# 얼굴 분류기 
### _ 무한도전 중 승리

### GitHub 모델 이용 https://github.com/davisking/dlib-models 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import dlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import tensorflow.keras 
from tensorflow.keras import backend as K

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat')

In [4]:
def find_faces(img):
    dets = detector(img, 1)
    
    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        # convert dlib shape to numpy array
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np

def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)

In [5]:
img_paths = {
     '승리one': 'del_img/victory1.jpg',
     '승리two': 'del_img/victory2.jpg',
     '탑one': 'del_img/top1.jpg',
     '탑two': 'del_img/top2.jpg',
}

descs = []

for name, img_path in img_paths.items(): 
    img = cv2.imread(img_path)
    _, img_shapes, _ = find_faces(img)
    descs.append([name, encode_faces(img, img_shapes)[0]])
    
np.save('user_img/descs.npy', descs)
print(descs)

[['승리one', array([-0.0381131 ,  0.08300128,  0.09253457, -0.01210379, -0.14962366,
        0.01980155, -0.07829851, -0.12751494,  0.08285795, -0.0882291 ,
        0.21364689, -0.03760978, -0.24343711, -0.08217979, -0.02008344,
        0.15759182, -0.06818017, -0.17166685, -0.02785353,  0.04396042,
        0.00627613,  0.04253965,  0.02330241,  0.01015794, -0.12500459,
       -0.32279682, -0.12047321, -0.04606272, -0.0579806 , -0.01137765,
       -0.12615696,  0.08817726, -0.11912832, -0.055033  ,  0.10761653,
        0.05045249, -0.00787995, -0.05086611,  0.20377606,  0.03319374,
       -0.24796316,  0.06600293,  0.06288414,  0.24162143,  0.18914346,
        0.00801563,  0.01462361, -0.11972698,  0.13268651, -0.12123196,
        0.08635782,  0.15799494,  0.16268079,  0.04324132,  0.03756319,
       -0.10500199, -0.01351237,  0.16270569, -0.14736851, -0.00564651,
        0.02949947, -0.06988099, -0.00505538, -0.1364612 ,  0.1336733 ,
        0.06657701, -0.08345699, -0.21515308,  0.1116

In [6]:
cam = cv2.VideoCapture("Images/기피대상.mp4") # 동영상
cam.set(3, 640) # 너비
cam.set(4, 480) # 높이
font = cv2.FONT_HERSHEY_SIMPLEX # 폰트

while True:
    
    ret, img = cam.read() 
    rects, shapes, _ = find_faces(img) # 얼굴 찾기
    descriptors = encode_faces(img, shapes) # 인코딩
    
    for i, desc in enumerate(descriptors):
        x = rects[i][0][0] # 얼굴 X 좌표
        y = rects[i][0][1] # 얼굴 Y 좌표
        w = rects[i][1][1]-rects[i][0][1] # 얼굴 너비 
        h = rects[i][1][0]-rects[i][0][0] # 얼굴 높이
        
        # 추출된 랜드마크와 데이터베이스의 랜드마크들 중 제일 짧은 거리를 찾는 부분
        descs1 = sorted(descs, key=lambda x: np.linalg.norm([desc] - x[1]))
        dist = np.linalg.norm([desc] - descs1[0][1], axis=1)
        
        if dist < 0.45: # 그 거리가 0.45보다 작다면 그 사람으로 판단 
            mosaic_img = cv2.resize(img[y:y+h, x:x+w], dsize=(0, 0), fx=0.04, fy=0.04) # 축소
            mosaic_img = cv2.resize(mosaic_img, (w, h), interpolation=cv2.INTER_AREA)  # 확대
            img[y:y+h, x:x+w] = mosaic_img

#     한글
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    img = np.array(img)
        
    cv2.imshow('camera', img)
    
    k = cv2.waitKey(10) & 0xff # 'ESC' 키 누르면 종료
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()